# Basic Llama communication

In [1]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

In [4]:
#callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q4_K_M.gguf",
    max_tokens=100000)

#input1 = "What is an atom? Answer using following structure: 1. DEFINITION, definition here, 2. SOURCES, sources here"
#output = llm.generate([input1])
#output.generations[0][0].text

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [35]:
input1 = "What is an atom? Answer using following structure: 1. DEFINITION, definition here, 2. SOURCES, sources here"

In [36]:
output = llm.generate([input1])

Llama.generate: prefix-match hit


In [37]:
output.generations[0][0].text

", 3. COMPOSITION, composition of an atom including protons, neutrons and electrons, 4. PROPERTIES, properties of an atom, 5. TYPES OF ATOMS, different types of atoms, 6. APPLICATIONS, applications of atoms in everyday life, 7. FUTURE, future of atoms, 8. CONCLUSION, conclusion about the importance of atoms.\n1. DEFINITION: An atom is the smallest unit of matter that retains the properties of an element. It consists of a nucleus and electrons orbiting around it. The nucleus contains protons and neutrons, while the electrons are negatively charged particles that orbit the nucleus in energy levels or electron shells.\n2. SOURCES: Atoms were first proposed by ancient Greek philosophers such as Democritus and Epicurus, but the modern understanding of atoms developed gradually over the 19th century through experiments and observations by scientists such as John Dalton, J.J. Thomson, and Ernest Rutherford.\n3. COMPOSITION: An atom consists of three main parts: protons, neutrons, and electron

In [2]:
prompt_template = PromptTemplate.from_template("What is the capital of {country}?")
prompt = prompt_template.format(country="France")
print(prompt)

What is the capital of France?


In [6]:
chain = prompt_template | llm

In [7]:
chain.schema()

{'title': 'RunnableSequence',
 'description': "A sequence of runnables, where the output of each is the input of the next.\n\nRunnableSequence is the most important composition operator in LangChain as it is\nused in virtually every chain.\n\nA RunnableSequence can be instantiated directly or more commonly by using the `|`\noperator where either the left or right operands (or both) must be a Runnable.\n\nAny RunnableSequence automatically supports sync, async, batch.\n\nThe default implementations of `batch` and `abatch` utilize threadpools and\nasyncio gather and will be faster than naive invocation of invoke or ainvoke\nfor IO bound runnables.\n\nBatching is implemented by invoking the batch method on each component of the\nRunnableSequence in order.\n\nA RunnableSequence preserves the streaming properties of its components, so if all\ncomponents of the sequence implement a `transform` method -- which\nis the method that implements the logic to map a streaming input to a streaming\no

In [15]:
llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q4_K_M.gguf",
    max_tokens=200000)
prompt_template = PromptTemplate.from_template("What is the capital of {country}?")
chain = prompt_template | llm
result = await chain.ainvoke({'country': 'France'})
result

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 
C:\Users\siwiakma\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\langchain\llms\llamacpp.py:352: RuntimeWarning: coroutine 'AsyncCallbackManagerForLLMRun.on_llm_new_token' was never awaited
  run_manager.on_llm_new_token(


# Using search 

## Prompting

In [1]:
prompt_template_s = """
<s>[INST] <<SYS>>
Hi, we will engage in an iterative, multi-step communication. 
You will be able to utilize external tools as needed for enhanced interaction, too! 
Available tools: {tool_descriptions}

Format for each iteration:
- Discussed issue: Central topic
- Line of thinking: Refine your response through multiple iterations, building upon previous insights.
- Action: Select one tool from {tool_names}. IMPORTANT: Please, here you must provide just a single word, the tool's name!
- Action Input: Provide input for the tool (e.g., search query for the 'Internet' tool). IMPORTANT: Please, you must provide just the input to be used, without any commentary! 
- Observations so far: Summarize and critically analyze information gathered.
Repeat the above steps as needed for a thorough exploration and conclusion.
- Final thought: Use this keyword only, when you'll complete your analysis after last internal iteration, otherwise don't use it! Craft a comprehensive, insightful communicative output.
<</SYS>>

Discussed issue: {text_input}
Communication hitory:
{agent_scratchpad} [/INST]
"""

In [4]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import LLMSingleActionAgent
from langchain.tools import Tool
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.utilities import SerpAPIWrapper


prompt_template_s = """
<s>[INST] <<SYS>>
Hi, we will engage in an iterative, multi-step communication. 
You will be able to utilize external tools as needed for enhanced interaction. 
Available tools: {tool_descriptions}

You MUST use one of the two versions of the following format for your answers! Do not provide any additional comments.
Version 1: You MUST use it, if you consider that your answer will be better if additional information of processing will be performed. 
- Discussed issue: Write here the central topic of the conversation. 
- Line of thinking: Present your line of thinking and steps you will in following iterations.
- Action: Select one tool from {tool_names}. IMPORTANT: Please, here you must provide just a single word, the tool's name!
- Action Input: Provide input for the tool (e.g., search query for the 'Internet' tool). IMPORTANT: Please, you must provide just the input to be used, without any commentary! 
- Observations so far: Summarize and critically analyze information gathered.
You will be provided with results of your information gathering action for the next iteration.

Version 2: Use it EXCLUSIVELY when you are sure that you got all the required information to fomulate the final opinion on the discussed topic: 
- Discussed issue: Write here the central topic of the conversation.
- Final thought (You must use this keyword ONLY, when you'll complete your analysis after last internal iteration!):  Craft a comprehensive, insightful communicative output.
<</SYS>>

Discussed issue: {text_input}
Communication hitory:
{agent_scratchpad} [/INST]
"""

llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q6_K.gguf",
    max_tokens=500000)

prompt_template = PromptTemplate.from_template(prompt_template_s)
chain = LLMChain(prompt=prompt_template, llm=llm, verbose=True)

search = SerpAPIWrapper(serpapi_api_key='d38482db0efe9c48ec77ffdd165c3fb4d7144a6bf86b3604e4367b0a0fe17fdd')
search_tool = Tool(
    name="Internet",
    func=search.run,
    description="Tool purpose: Search web for e.g., current events and state of affairs.; Tool syntax (Action Input): your_search_query"
)
fake_tool = Tool(
    name="Fake",
    func=search.run,
    description="Tool purpose: It serves not other purpose than to confuse.; Tool syntax (Action Input): Whatever, it doesn't do anything anyway"
)

tools = [search_tool, fake_tool]
tool_description_s = "\n".join(
    [f"\n{index + 1}. Tool name (Action): {tool.name}; Tool details: {tool.description}" 
     for index, tool in enumerate(tools)]
)
tool_names_s = "[" + ", ".join([tool.name for tool in tools]) + "]"

text_input = "What are the latest advancements in quantum computing as of November 2023?" 

filled_prompt = prompt_template_s.format(
    tool_descriptions=tool_description_s,
    tool_names=tool_names_s,
    text_input=text_input,
    agent_scratchpad={}
)

print(llm.generate(prompts=[filled_prompt]))

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


generations=[[Generation(text='Line of thinking:\n1. Research current events and state of affairs related to quantum computing.\n2. Analyze')]] llm_output=None run=[RunInfo(run_id=UUID('6454d4d5-01c0-48e6-a31a-6d208020d773'))]


# Output parsing

In [17]:
class ConversationStorage:
    def __init__(self, cognitive_feedback_router_instance):
        # Store conversation data with discussion topic as the main key
        self.conversation_data = defaultdict(lambda: defaultdict(dict))
        self.conversation_history = []
        self.cognitive_feedback_router = cognitive_feedback_router_instance
    
    def update_conversation(self, discussion_topic, iteration_text):
        # Parse the current iteration
        parsed_iteration = self.flexible_parse_iteration(iteration_text)
        # Determine the iteration number for the current topic
        iteration_number = len(self.conversation_data[discussion_topic]) + 1
        
        # Add the parsed iteration to the conversation data
        self.conversation_data[discussion_topic][iteration_number] = parsed_iteration
        
        # Add the raw text to the conversation history
        self.conversation_history.append(iteration_text)

        # Decide the action based on parsed data
        if "Action" in parsed_iteration and "Action Input" in parsed_iteration:
            # Call TakeAction's reaction method
            TakeAction.reaction(parsed_iteration["Action"], parsed_iteration["Action Input"])
        elif "Final thought" in parsed_iteration:
            print(parsed_iteration["Final thought"])
            # Pass to CognitiveFeedbackRouter's display method
            self.cognitive_feedback_router.display(parsed_iteration["Final thought"])
            # Send the entire agent_scratchpad to NeuralGraphBridge
            NeuralGraphBridge.conversation(self.format_for_scratchpad())
    
    def add_result(self, discussion_topic, result):
        # Identify the highest iteration number for the given discussion topic
        if discussion_topic in self.conversation_data:
            highest_iteration = max(self.conversation_data[discussion_topic])
            # Add the result to the latest iteration of the given discussion topic
            self.conversation_data[discussion_topic][highest_iteration]['Action result'] = result
        else:
            # If the topic is not found, this function does nothing (or you can handle it differently)
            pass

    def format_for_scratchpad(self):
        scratchpad_text = ""
        for topic, iterations in self.conversation_data.items():
            scratchpad_text += f"Discussion Topic: {topic}\n"
            for iteration, data in iterations.items():
                scratchpad_text += f"  Iteration {iteration}:\n"
                for key, value in data.items():
                    scratchpad_text += f"    {key}: {value}\n"
            scratchpad_text += "\n"
        return scratchpad_text.strip()

    @staticmethod
    def flexible_parse_iteration(iteration_text):
        # Labels to look for
        labels = ["Discussed issue", "Line of thinking", "Action", "Action Input", "Observations so far", "Final thought"]

        # Initialize a dictionary to store the parsed data for this iteration
        agent_scratchpad = {}

        # Process each label
        for label in labels:
            # Find the element text using regex, capturing until the next label or the end of the iteration
            pattern = f"{label}: ([\\s\\S]*?)(?=(" + "|".join(labels) + "|$))"
            match = re.search(pattern, iteration_text)

            if match:
                agent_scratchpad[label] = match.group(1).strip()
            else:
                # If the label is not found, check if its content might be under a different label
                for alt_label in labels:
                    if alt_label != label and alt_label in agent_scratchpad:
                        # Check if the content for the alt_label seems to mistakenly include the content for the current label
                        if f"{label}:" in agent_scratchpad[alt_label]:
                            split_content = agent_scratchpad[alt_label].split(f"{label}:")
                            # Assume the first part is for the alt_label and the second part (if exists) is for the current label
                            agent_scratchpad[alt_label] = split_content[0].strip()
                            if len(split_content) > 1:
                                agent_scratchpad[label] = split_content[1].strip()
                                break

        return agent_scratchpad


# Assuming TakeAction, CognitiveFeedbackRouter, and NeuralGraphBridge are defined elsewhere
class TakeAction:
    @staticmethod
    def reaction(action, action_input):
        # Implementation of the reaction method
        pass

class CognitiveFeedbackRouter:
    def display(self, final_thought):
        # Implementation of the display method
        pass

class NeuralGraphBridge:
    @staticmethod
    def conversation(agent_scratchpad):
        # Implementation of the conversation method
        pass

# Example instantiation of CognitiveFeedbackRouter and ConversationStorage
cognitive_feedback_router_instance = CognitiveFeedbackRouter()

# Example usage
conversation_store = ConversationStorage(cognitive_feedback_router_instance)

# Simulate adding iterations to the conversation
conversation_store.update_conversation("Climate Change", "Discussed issue: Climate change effects on agriculture. Line of thinking: Research data. Action: Internet. Action Input: Search for recent data. Observations so far: Significant impacts observed.")
conversation_store.add_result("Climate Change", "Found relevant data on climate impact.")
conversation_store.update_conversation("Education", "Discussed issue: Technology in education. Final thought: Critical role and challenges.")

# Format the conversation for the scratchpad
scratchpad_formatted = conversation_store.format_for_scratchpad()
print(scratchpad_formatted)

Critical role and challenges.
Discussion Topic: Climate Change
  Iteration 1:
    Discussed issue: Climate change effects on agriculture.
    Line of thinking: Research data.
    Action: Internet.
    Action Input: Search for recent data.
    Observations so far: Significant impacts observed.
    Action result: Found relevant data on climate impact.

Discussion Topic: Education
  Iteration 1:
    Discussed issue: Technology in education.
    Final thought: Critical role and challenges.


# Graphs

In [18]:
from langchain.graphs import Neo4jGraph

In [19]:
graph = Neo4jGraph(
    url="bolt://localhost:7687", 
    username="neo4j", 
    password="Ne5Invoces"
)

In [20]:
graph.query(
"""
MERGE (m:Movie {name:"Top Gun"})
WITH m
UNWIND ["Tom Cruise", "Val Kilmer", "Anthony Edwards", "Meg Ryan"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)
"""
)

[]

In [21]:
graph.refresh_schema()
print(graph.schema)


        Node properties are the following:
        [{'labels': 'Professor', 'properties': [{'property': 'name', 'type': 'STRING'}, {'property': 'department', 'type': 'STRING'}]}, {'labels': 'Student', 'properties': [{'property': 'name', 'type': 'STRING'}, {'property': 'major', 'type': 'STRING'}]}, {'labels': 'Course', 'properties': [{'property': 'code', 'type': 'STRING'}, {'property': 'name', 'type': 'STRING'}]}, {'labels': 'Movie', 'properties': [{'property': 'name', 'type': 'STRING'}]}, {'labels': 'Actor', 'properties': [{'property': 'name', 'type': 'STRING'}]}]
        Relationship properties are the following:
        []
        The relationships are the following:
        ['(:Professor)-[:TEACHES]->(:Course)', '(:Student)-[:ENROLLED_IN]->(:Course)', '(:Actor)-[:ACTED_IN]->(:Movie)']
        


In [22]:
Neo4jGraph.__dict__

mappingproxy({'__module__': 'langchain.graphs.neo4j_graph',
              '__doc__': 'Neo4j wrapper for graph operations.\n\n    *Security note*: Make sure that the database connection uses credentials\n        that are narrowly-scoped to only include necessary permissions.\n        Failure to do so may result in data corruption or loss, since the calling\n        code may attempt commands that would result in deletion, mutation\n        of data if appropriately prompted or reading sensitive data if such\n        data is present in the database.\n        The best way to guard against such negative outcomes is to (as appropriate)\n        limit the permissions granted to the credentials used with this tool.\n\n        See https://python.langchain.com/docs/security for more information.\n    ',
              '__init__': <function langchain.graphs.neo4j_graph.Neo4jGraph.__init__(self, url: Union[str, NoneType] = None, username: Union[str, NoneType] = None, password: Union[str, NoneType] =

In [58]:
from langchain.graphs.graph_document import Node, Relationship, GraphDocument
from langchain.schema import Document

def create_graph_document_from_dict(data_dict):
    # Create Node objects from 'nodes' in the dictionary
    nodes = [Node(id=node_id, properties=props) for node_id, props in data_dict['nodes'].items()]

    # Create a dictionary for quick node lookup by ID
    nodes_lookup = {node.id: node for node in nodes}

    # Create Relationship objects from 'edges' in the dictionary
    edges = []
    for (source_id, target_id), edge_info in data_dict['edges'].items():
        # Check if both nodes exist
        if source_id in nodes_lookup and target_id in nodes_lookup:
            relationship_type = edge_info.pop('type', 'RELATES_TO')  # Extract the 'type'
            edge = Relationship(
                source=nodes_lookup[source_id], 
                target=nodes_lookup[target_id], 
                type=relationship_type, 
                properties=edge_info
            )
            edges.append(edge)

    # Extract 'page_content' and 'metadata' for the Document object
    page_content = data_dict['metadata'].pop('page_content', '')
    metadata = data_dict['metadata']

    # Create a Document object
    source_document = Document(
        page_content=page_content,
        metadata=metadata
    )

    # Create and return a GraphDocument object
    return GraphDocument(
        nodes=nodes,
        relationships=edges,
        source=source_document
    )

In [59]:
graph_document = create_graph_document_from_dict(example_dict)

In [60]:
graph_document

GraphDocument(nodes=[Node(id='node_name1', properties={'name_parameter1': 'parameter1_value', 'name_parameter2': 'parameter2_value'}), Node(id='node_name2', properties={'name_parameter1': 'parameter1_value', 'name_parameter3': 'parameter3_value'})], relationships=[Relationship(source=Node(id='node_name1', properties={'name_parameter1': 'parameter1_value', 'name_parameter2': 'parameter2_value'}), target=Node(id='node_name2', properties={'name_parameter1': 'parameter1_value', 'name_parameter3': 'parameter3_value'}), type='RELATES_TO', properties={'name_parameter4': 'parameter4_value', 'name_parameter6': 'parameter2_value'})], source=Document(page_content='', metadata={'date': 'Some date', 'Interactee': 'Person', 'Tone': 'Somber', 'Urgency': 'Low'}))

In [62]:
graph.add_graph_documents(graph_documents=[graph_document], include_source=True)